## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [1]:
import pandas as pd

file_path = 'Resources/client_dataset.csv'
data = pd.read_csv(file_path)

In [3]:
# View the column names in the data
print("Column Names:")
print(data.columns)

Column Names:
Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')


In [5]:
# Use the describe function to gather some basic statistics
print("\nBasic Statistics:")
print(data.describe())


Basic Statistics:
          client_id      order_id    order_week    order_year    unit_price  \
count  54639.000000  5.463900e+04  54639.000000  54639.000000  54639.000000   
mean   54837.869416  5.470190e+06     11.359139   2022.993064    136.267207   
std    25487.438231  2.599807e+06      7.023499      0.082997    183.873135   
min    10033.000000  1.000886e+06      1.000000   2022.000000      0.010000   
25%    33593.000000  3.196372e+06      6.000000   2023.000000     20.800000   
50%    53305.000000  5.496966e+06     11.000000   2023.000000     68.310000   
75%    78498.000000  7.733869e+06     17.000000   2023.000000    173.160000   
max    99984.000000  9.998480e+06     52.000000   2023.000000   1396.230000   

          unit_cost   unit_weight           qty   line_number  
count  54639.000000  54639.000000  5.463900e+04  54639.000000  
mean      99.446073      5.004116  5.702646e+02      2.979667  
std      133.164267      5.326599  1.879552e+04      2.436320  
min        0.

In [7]:
# Use this space to do any additional research
# and familiarize yourself with the data.
['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id', 'order_date', 'order_week', 'order_year', 'item_id', 'category', 'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty', 'line_number']


['first',
 'last',
 'job',
 'phone',
 'email',
 'client_id',
 'order_id',
 'order_date',
 'order_week',
 'order_year',
 'item_id',
 'category',
 'subcategory',
 'unit_price',
 'unit_cost',
 'unit_weight',
 'qty',
 'line_number']

In [9]:
# What three item categories had the most entries?
top_3_categories = data['category'].value_counts().head(3)

In [11]:
# For the category with the most entries, which subcategory had the most entries?
top_category = top_3_categories.index[0]
top_subcategory = data[data['category'] == top_category]['subcategory'].value_counts().idxmax()


In [13]:
# Which five clients had the most entries in the data?
top_5_clients = data['client_id'].value_counts().head(5)

In [15]:
# Store the client ids of those top 5 clients in a list.
top_5_client_ids = top_5_clients.index.tolist()

In [48]:
# How many total units (the qty column) did the client with the most entries order order?
top_client_id = top_5_client_ids[0]
total_units_top_client = data[data['client_id'] == top_client_id]['qty'].sum()

top_3_categories, top_subcategory, top_5_clients, top_5_client_ids, total_units_top_client

(category
 consumables    23538
 furniture      11915
 software        8400
 Name: count, dtype: int64,
 'bathroom supplies',
 client_id
 33615    220
 66037    211
 46820    209
 38378    207
 24741    207
 Name: count, dtype: int64,
 [33615, 66037, 46820, 38378, 24741],
 64313)

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [51]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
data['subtotal'] = data['unit_price'] * data['qty']

In [53]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.
data['shipping_price'] = data['unit_weight'].apply(lambda x: 7 if x > 50 else 10) * data['unit_weight']


In [55]:
# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%
data['total_price'] = data['subtotal'] + data['shipping_price']
data['total_price'] += data['total_price'] * 0.0925

In [57]:
# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).
data['line_cost'] = (data['unit_cost'] * data['qty']) + data['shipping_price']


In [59]:
# Create a column for the profit of each line using line cost and line price
data['profit'] = data['total_price'] - data['line_cost']


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [62]:
# Check your work using the totals above
order_ids_to_confirm = [2742071, 2173913, 6128929]
confirmed_prices = data[data['order_id'].isin(order_ids_to_confirm)].groupby('order_id')['total_price'].sum()

confirmed_prices

order_id
2173913    119838.357050
2742071    139844.512025
6128929    752748.745475
Name: total_price, dtype: float64

## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [65]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.
top_5_client_ids = [33615, 66037, 46820, 38378, 24741]

In [67]:
# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit. 
top_5_summary = data[data['client_id'].isin(top_5_client_ids)].groupby('client_id').agg({
    'qty': 'sum',
    'shipping_price': 'sum',
    'total_price': 'sum',
    'profit': 'sum'
}).reset_index()

In [ ]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
def to_millions(amount):
    return amount / 1e6